In [12]:
import pandas as pd
import pymysql.cursors

In [13]:
connection = pymysql.connect(host='localhost', user='root', password='breadt@2019', db='breadt-football-ml', charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)
with connection.cursor() as cursor:
    sql = "select game, issue,matchid from `breadt_lottery_info` where time > '2018-01-01';"
    cursor.execute(sql)
    rows = cursor.fetchall()
    
    lottery_df = pd.DataFrame(rows)
    
    connection.close()

In [14]:
lottery_df[lottery_df['issue'] == 19045]

,game,issue,matchid
2995,英超,19045,2405563
2996,英超,19045,2405516
2997,意甲,19045,2437944
2998,意甲,19045,2437942
2999,意甲,19045,2437937
3000,意甲,19045,2437940
3001,意甲,19045,2437895
3002,意甲,19045,2437941
3003,意甲,19045,2437939
3004,意甲,19045,2437943


In [15]:
result_df = pd.read_csv('../data/submission_2019.csv')

In [16]:
df = pd.merge(lottery_df, result_df, on='matchid', how='left')
df = df.dropna()
df = df.drop_duplicates()

In [17]:
df[df['issue']==19045]

,game_x,issue,matchid,game_y,result,pred,time
2995,英超,19045,2405563,英超,2.0,2.0,2019-04-03 00:00:00
2996,英超,19045,2405516,英超,2.0,2.0,2019-04-04 00:00:00
2997,意甲,19045,2437944,意甲,1.0,1.0,2019-04-03 00:00:00
2998,意甲,19045,2437942,意甲,0.0,0.0,2019-04-03 00:00:00
2999,意甲,19045,2437937,意甲,2.0,2.0,2019-04-04 00:00:00
3000,意甲,19045,2437940,意甲,1.0,1.0,2019-04-04 00:00:00
3001,意甲,19045,2437895,意甲,2.0,2.0,2019-04-04 00:00:00
3002,意甲,19045,2437941,意甲,2.0,0.0,2019-04-04 03:00:00
3003,意甲,19045,2437939,意甲,0.0,0.0,2019-04-04 00:00:00
3004,意甲,19045,2437943,意甲,2.0,2.0,2019-04-04 03:00:00


In [18]:
df['score'] = df.apply(lambda row: 1 if row.result == row.pred else 0, axis=1)
df['exist'] = 1

In [19]:
result_df[result_df['matchid'] == 2405531]

,game,matchid,result,pred,time
1412,英超,2405531,2,2.0,2019-03-30 23:00:00


In [20]:
df[df['issue'] == 19061]

,game_x,issue,matchid,game_y,result,pred,time,score,exist
3223,英冠,19061,2411095,英冠,0.0,1.0,2019-05-01 00:00:00,0,1
3224,英冠,19061,2411100,英冠,1.0,0.0,2019-05-02 00:00:00,0,1
3225,法甲,19061,2402946,法甲,2.0,2.0,2019-05-01 00:00:00,1,1
3226,法甲,19061,2402948,法甲,1.0,2.0,2019-05-02 00:00:00,0,1
3227,巴甲,19061,2529128,巴甲,2.0,2.0,2019-05-02 00:00:00,1,1
3228,巴甲,19061,2529126,巴甲,2.0,2.0,2019-05-02 00:00:00,1,1
3229,巴甲,19061,2529124,巴甲,1.0,2.0,2019-05-02 00:00:00,0,1
3230,巴甲,19061,2529132,巴甲,2.0,2.0,2019-05-02 00:00:00,1,1
3231,巴甲,19061,2529130,巴甲,0.0,2.0,2019-05-02 00:00:00,0,1
3232,巴甲,19061,2529127,巴甲,0.0,2.0,2019-05-02 00:00:00,0,1


In [21]:
t = df
final_result = t.groupby(t['issue'])[['score','exist']].sum()

In [22]:
f = final_result[final_result['exist'] > 0]
f.sort_values(by=['score', 'exist'], ascending=False)

,score,exist
issue,,
19055,14,14
19051,13,14
19031,12,14
19032,12,14
19043,12,14
19045,12,14
19047,12,14
19058,12,14
19028,11,14


+ 19038 - 19041 都是国家队比赛 应该是世界杯
+ 19030 19034 全部是亚赛和欧赛
